In [15]:
# Dependencies
import requests
import xml.etree.ElementTree as ET # to parse XML
import csv
from config import API_KEY

import numpy as np
import pandas as pd

# API source : GreatSchools
https://www.greatschools.org/api/docs/school-profile/
response is returned in XML format.

* XML format explanation : https://www.w3schools.com/xml/xml_tree.asp
* cf. parsing example : https://www.datacamp.com/community/tutorials/python-xml-elementtree

## testing with CA

In [45]:
# URL prep
URL = 'https://api.greatschools.org/schools/CA/"*"?'
queryURL = URL + 'key=' + API_KEY + "&limit=50000"

print(queryURL)
# https://api.greatschools.org/search/schools?key=[yourAPIKey]&state=CA&q=Alameda&sort=alpha&levelCode=elementary-schools&limit=10

In [47]:
# request
response = requests.get(queryURL)
# response.content

In [48]:
# parse
tree = ET.fromstring(response.content)

df = pd.DataFrame()
schoolProfile={}

for school in tree:
    for child in school:
#         print(child.tag, child.text)
        schoolProfile[str(child.tag)] = str(child.text)
    df = df.append(schoolProfile, ignore_index=True)

print(df.columns)
print(len(df))
print(df.dtypes)
df.head()

Index(['address', 'city', 'district', 'districtId', 'districtNCESId',
       'enrollment', 'fax', 'gradeRange', 'gsId', 'gsRating', 'lat', 'lon',
       'name', 'ncesId', 'overviewLink', 'parentRating', 'phone',
       'ratingsLink', 'reviewsLink', 'schoolStatsLink', 'state', 'type',
       'website'],
      dtype='object')
15762
address            object
city               object
district           object
districtId         object
districtNCESId     object
enrollment         object
fax                object
gradeRange         object
gsId               object
gsRating           object
lat                object
lon                object
name               object
ncesId             object
overviewLink       object
parentRating       object
phone              object
ratingsLink        object
reviewsLink        object
schoolStatsLink    object
state              object
type               object
website            object
dtype: object


,address,city,district,districtId,districtNCESId,enrollment,fax,gradeRange,gsId,gsRating,...,ncesId,overviewLink,parentRating,phone,ratingsLink,reviewsLink,schoolStatsLink,state,type,website
0,"4610 South Main Street, \nLos Angeles, CA 90037",Los Angeles,Los Angeles Unified School District,286,0622710,630,(213) 943-4931,9-12,17662,7,...,062271011632,https://www.greatschools.org/california/los-an...,5,(213) 342-2874,https://www.greatschools.org/california/los-an...,https://www.greatschools.org/california/los-an...,https://www.greatschools.org/california/los-an...,CA,charter,http://collegeready5.org
1,"9753 Ellington Lane, \nTruckee, CA 96161",Truckee,Los Angeles Unified School District,286,0622710,25,None,9-12,32348,7,...,None,https://www.greatschools.org/california/trucke...,5,(530) 536-5333,https://www.greatschools.org/california/trucke...,https://www.greatschools.org/california/trucke...,https://www.greatschools.org/california/trucke...,CA,private,None
2,"939 Michigan Avenue, \nBeaumont, CA 92223",Beaumont,Beaumont Unified School District,515,0604290,100,None,K-12,31844,2,...,060429013779,https://www.greatschools.org/california/beaumo...,5,(951) 769-8424,https://www.greatschools.org/california/beaumo...,https://www.greatschools.org/california/beaumo...,https://www.greatschools.org/california/beaumo...,CA,public,http://glenview-beaumont-ca.schoolloop.com/21s...
3,"4138 Lakeside Drive, \nRichmond, CA 94806",Richmond,Beaumont Unified School District,515,0604290,29,(510) 262-1540,K-12,13424,2,...,K9300854,https://www.greatschools.org/california/richmo...,4,(510) 262-1500,https://www.greatschools.org/california/richmo...,https://www.greatschools.org/california/richmo...,https://www.greatschools.org/california/richmo...,CA,private,http://www.calautism.org/programs/abc-school/
4,"4138 Lakeside Drive, \nRichmond, CA 94806",Richmond,Beaumont Unified School District,515,0604290,37,None,4-12,24131,2,...,A0500119,https://www.greatschools.org/california/richmo...,4,(510) 262-1500,https://www.greatschools.org/california/richmo...,https://www.greatschools.org/california/richmo...,https://www.greatschools.org/california/richmo...,CA,private,None


In [49]:
# select/sort columns and drop unnecessary columns.
df_selected = df[['gsId','name','type','gradeRange','enrollment','parentRating','gsRating',
         'district', 'districtId', 'districtNCESId','ncesId',
         'address','city','state','lat','lon','phone']]

In [50]:
# convert to numeric
try:
    df_selected['gsId'] = pd.to_numeric(df_selected['gsId'])
    df_selected['enrollment'] = pd.to_numeric(df_selected['enrollment'])
    df_selected['parentRating'] = pd.to_numeric(df_selected['parentRating'])
    df_selected['gsRating'] = pd.to_numeric(df_selected['gsRating'])
    df_selected['districtId'] = pd.to_numeric(df_selected['districtId'])
    df_selected['districtNCESId'] = pd.to_numeric(df_selected['districtNCESId'])
    df_selected['ncesId'] = pd.to_numeric(df_selected['ncesId'])
    df_selected['lat'] = pd.to_numeric(df_selected['lat'])
    df_selected['lon'] = pd.to_numeric(df_selected['lon'])
except (ValueError):
    print("Unable to parse/convert... skipping.")

C:\Users\soyou\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\soyou\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\soyou\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

Unable to parse/convert... skipping.


C:\Users\soyou\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [51]:
# sort by 'gsID'
df_sorted=df_selected.sort_values(["gsId"])
df_sorted.head()

,gsId,name,type,gradeRange,enrollment,parentRating,gsRating,district,districtId,districtNCESId,ncesId,address,city,state,lat,lon,phone
244,1,Alameda High School,public,9-12,1853,4,9,Alameda Unified School District,1,0601770,060177000041,"2201 Encinal Avenue, \nAlameda, CA 94501",Alameda,CA,37.764294,-122.24807,(510) 337-7022
1087,2,Bay Farm,public,K-8,591,5,9,Alameda Unified School District,1,0601770,060177000006,"200 Aughinbaugh Way, \nAlameda, CA 94502",Alameda,CA,37.739,-122.25195,(510) 748-4010
495,4,Amelia Earhart Elementary School,public,K-5,622,5,9,Alameda Unified School District,1,0601770,060177000043,"400 Packet Landing Road, \nAlameda, CA 94502",Alameda,CA,37.74529,-122.23928,(510) 748-4003
3952,5,Edison Elementary School,public,K-5,478,4,9,Alameda Unified School District,1,0601770,060177000044,"2700 Buena Vista Avenue, \nAlameda, CA 94501",Alameda,CA,37.765083,-122.23421,(510) 748-4002
4212,6,Encinal Junior/Senior High,public,6-12,1336,4,6,Alameda Unified School District,1,0601770,060177000045,"210 Central Avenue, \nAlameda, CA 94501",Alameda,CA,37.772766,-122.289,(510) 748-4023


In [52]:
# drop duplicates, keep one row
df_sorted.drop_duplicates(subset='gsId', keep='last', inplace=True)
print(len(df_sorted))

15762


In [53]:
df_clean = df_sorted.dropna()
print(len(df_clean))

15762


In [54]:
df_sorted.to_csv('GreatSchools_CA.csv',index=False)

<hr></hr>

## testing to collect all states' data

In [11]:
# URL prep
URL = 'https://api.greatschools.org/schools/'
states=['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA',
        'HI','ID','IL','IN','IA','KS','KY','LA','ME','MD',
        'MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ',
        'NM','NY','NC','ND','OH','OK','OR','PA','RI','SC',
        'SD','TN','TX','UT','VT','VA','WA','WV','WI','WY'
       ]
# print(len(states))

queryURL = f"{URL}{state}/'*'?key={API_KEY}"
# print(queryURL)

NameError: name 'state' is not defined

In [14]:
df_all = pd.DataFrame()
schoolProfile_all={}

for state in states:
    queryURL = f'{URL}{state}/"*"?key={API_KEY}'
    response = requests.get(queryURL)
#     print(queryURL)
    tree = ET.fromstring(response.content)
#     print(tree)

    for school in tree:
        for child in school:
#             print(child.tag, child.text)
            schoolProfile_all[str(child.tag)] = str(child.text)
        df_all = df_all.append(schoolProfile_all, ignore_index=True)

print(df_all.columns)
print(len(df_all))
print(df_all.dtypes)
print(len(df_all))
df_all.head()
    

Index(['address', 'city', 'fax', 'gradeRange', 'gsId', 'lat', 'lon', 'name',
       'ncesId', 'overviewLink', 'phone', 'ratingsLink', 'reviewsLink',
       'schoolStatsLink', 'state', 'type', 'website', 'district', 'districtId',
       'districtNCESId', 'enrollment', 'gsRating', 'parentRating'],
      dtype='object')
2000
address            object
city               object
fax                object
gradeRange         object
gsId               object
lat                object
lon                object
name               object
ncesId             object
overviewLink       object
phone              object
ratingsLink        object
reviewsLink        object
schoolStatsLink    object
state              object
type               object
website            object
district           object
districtId         object
districtNCESId     object
enrollment         object
gsRating           object
parentRating       object
dtype: object
2000


,address,city,fax,gradeRange,gsId,lat,lon,name,ncesId,overviewLink,...,schoolStatsLink,state,type,website,district,districtId,districtNCESId,enrollment,gsRating,parentRating
0,"838 W Main St, \nPrichard, AL 36610",Prichard,None,6-12,3708,30.736015,-88.09117,100 BLK Men of Greater Mobile's Phoenix Program,A0900001,https://www.greatschools.org/alabama/prichard/...,...,https://www.greatschools.org/alabama/prichard/...,AL,private,None,NaN,NaN,NaN,NaN,NaN,NaN
1,"626 North Daleville Ave, \nDaleville, AL 36322",Daleville,(334) 598-9006,K-6,468,31.310625,-85.71681,A M Windham Elementary School,010108001767,https://www.greatschools.org/alabama/daleville...,...,https://www.greatschools.org/alabama/daleville...,AL,public,http://www.daleville.k12.al.us/wes.html,Daleville City School District,45,0101080,539,5,3
2,"501 Beck St, \nAtmore, AL 36502",Atmore,(251) 368-0244,3-4,565,31.015285,-87.485054,A.C. Moore Elementary School,010135000482,https://www.greatschools.org/alabama/atmore/56...,...,https://www.greatschools.org/alabama/atmore/56...,AL,public,http://www.escambiak12.net,Escambia County School District,54,0101350,280,3,1
3,"300a North Trawick St, \nAbbeville, AL 36310",Abbeville,(334) 585-2012,1-12,2247,31.567364,-85.25238,Abbeville Alt School,010174002120,https://www.greatschools.org/alabama/abbeville...,...,https://www.greatschools.org/alabama/abbeville...,AL,public,http://www.henrycountyboe.org,Henry County School District,70,0101740,280,3,1
4,"M L Tillis Dr, \nAbbeville, AL 36310",Abbeville,None,PK-12,1572,31.576424,-85.26009,Abbeville Christian Academy,00001026,https://www.greatschools.org/alabama/abbeville...,...,https://www.greatschools.org/alabama/abbeville...,AL,private,https://acagenerals.org/,Henry County School District,70,0101740,280,3,5
